In [6]:
import pandas as pd
import numpy as np

# Load files
churn = pd.read_csv("/content/telecom_customer_churn.csv")
dictionary = pd.read_csv("/content/telecom_data_dictionary.csv", encoding="latin1")
zipcode = pd.read_csv("/content/telecom_zipcode_population.csv")

print("RAW DATA:")
display(churn.head())

# Clean column names
churn.columns = churn.columns.str.lower().str.replace(" ", "_")
zipcode.columns = zipcode.columns.str.lower().str.replace(" ", "_")
print("\nAfter cleaning column names:")
display(churn.head())

# Merge ZIP population
if "zip_code" in churn.columns:
    churn = churn.merge(zipcode[["zip_code", "population"]], on="zip_code", how="left")
print("\nAfter merging ZIP population:")
display(churn.head())

# Remove duplicates
churn = churn.drop_duplicates()
print("\nAfter removing duplicates:")
display(churn.head())

# Replace invalid nulls
churn.replace(["?", "", "NA", "N/A", "null"], np.nan, inplace=True)
print("\nAfter replacing invalid nulls:")
display(churn.head())

# Fill numeric missing values
num_cols = churn.select_dtypes(include=[np.number]).columns
for col in num_cols:
    churn[col].fillna(churn[col].median(), inplace=True)
print("\nAfter filling numeric missing values:")
display(churn.head())

# Fill categorical missing values
cat_cols = churn.select_dtypes(include="object").columns
for col in cat_cols:
    churn[col].fillna(churn[col].mode()[0], inplace=True)
print("\nAfter filling categorical missing values:")
display(churn.head())

# Convert numeric-looking columns
for col in churn.columns:
    try:
        churn[col] = pd.to_numeric(churn[col])
    except:
        pass
print("\nAfter converting numeric-looking columns:")
display(churn.head())

# Detect and cap outliers
print("\nOutlier detection and handling:")
for col in num_cols:
    Q1 = churn[col].quantile(0.25)
    Q3 = churn[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = churn[(churn[col] < lower) | (churn[col] > upper)][col]
    print(f"\nColumn: {col}, Outliers: {len(outliers)}")
    if len(outliers) > 0:
        display(outliers.head())

    churn[col] = np.where(churn[col] < lower, lower,
                          np.where(churn[col] > upper, upper, churn[col]))

# Final cleaned data
print("\nFINAL CLEANED DATA:")
display(churn.head())
print("\nFinal shape:", churn.shape)


RAW DATA:


,Customer ID,Gender,Age,Married,Number of Dependents,City,Zip Code,Latitude,Longitude,Number of Referrals,...,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Customer Status,Churn Category,Churn Reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability



After cleaning column names:


,customer_id,gender,age,married,number_of_dependents,city,zip_code,latitude,longitude,number_of_referrals,...,payment_method,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue,customer_status,churn_category,churn_reason
0,0002-ORFBO,Female,37,Yes,0,Frazier Park,93225,34.827662,-118.999073,2,...,Credit Card,65.6,593.30,0.00,0,381.51,974.81,Stayed,NaN,NaN
1,0003-MKNFE,Male,46,No,0,Glendale,91206,34.162515,-118.203869,0,...,Credit Card,-4.0,542.40,38.33,10,96.21,610.28,Stayed,NaN,NaN
2,0004-TLHLJ,Male,50,No,0,Costa Mesa,92627,33.645672,-117.922613,0,...,Bank Withdrawal,73.9,280.85,0.00,0,134.60,415.45,Churned,Competitor,Competitor had better devices
3,0011-IGKFF,Male,78,Yes,0,Martinez,94553,38.014457,-122.115432,1,...,Bank Withdrawal,98.0,1237.85,0.00,0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction
4,0013-EXCHZ,Female,75,Yes,0,Camarillo,93010,34.227846,-119.079903,3,...,Credit Card,83.9,267.40,0.00,0,22.14,289.54,Churned,Dissatisfaction,Network reliability



After merging ZIP population:


...


Column: population, Outliers: 57


,population
108,93315
296,91664
358,105285
917,87383
919,97318



FINAL CLEANED DATA:


,customer_id,gender,age,married,number_of_dependents,city,zip_code,latitude,longitude,number_of_referrals,...,monthly_charge,total_charges,total_refunds,total_extra_data_charges,total_long_distance_charges,total_revenue,customer_status,churn_category,churn_reason,population
0,0002-ORFBO,Female,37.0,Yes,0.0,Frazier Park,93225.0,34.827662,-118.999073,2.0,...,65.6,593.30,0.0,0.0,381.51,974.81,Stayed,Competitor,Competitor had better devices,4498.0
1,0003-MKNFE,Male,46.0,No,0.0,Glendale,91206.0,34.162515,-118.203869,0.0,...,-4.0,542.40,0.0,0.0,96.21,610.28,Stayed,Competitor,Competitor had better devices,31297.0
2,0004-TLHLJ,Male,50.0,No,0.0,Costa Mesa,92627.0,33.645672,-117.922613,0.0,...,73.9,280.85,0.0,0.0,134.60,415.45,Churned,Competitor,Competitor had better devices,62069.0
3,0011-IGKFF,Male,78.0,Yes,0.0,Martinez,94553.0,38.014457,-122.115432,1.0,...,98.0,1237.85,0.0,0.0,361.66,1599.51,Churned,Dissatisfaction,Product dissatisfaction,46677.0
4,0013-EXCHZ,Female,75.0,Yes,0.0,Camarillo,93010.0,34.227846,-119.079903,3.0,...,83.9,267.40,0.0,0.0,22.14,289.54,Churned,Dissatisfaction,Network reliability,42853.0



Final shape: (7043, 39)
